### Read the Latency to startle.ipynb for an understanding of the filtering procedures. The calculation of %PPI uses the default Vmax and does not rectify based on preceding peaks.

In [ ]:
import pandas as pd
import numpy as np
import statistics as st
from statistics import StatisticsError
import plotly.express as plt
import scipy
from scipy.signal import find_peaks
from decimal import Decimal as D
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
from scipy.signal import chirp, find_peaks, peak_widths
import re

In [ ]:
# row 8 in big_df is the 1st ms; every row corresponds to an ms. The column corresponds to vmax values

In [ ]:
def ColumNames ():
    """No input needed
       Output : List with modified trial names as elements such as Trial3_Mouse4"""
    columnames = []
    Mouse_N = [1,2,3,4]*72
    Trial_N = ['Trial'+str(i) for i in range(1,73) for y in range (4)]
    for i in range (0,288):
        columnames.append(Trial_N[i]+'_Mouse'+str(Mouse_N[i]))
    return columnames

In [ ]:
def Find_Maxima_And_Exclude (bigdf, file, cut_criterion_V=25, cut_criterion_T=30):
    """ Input 1    : dataframe
        Input 2    : str indicating a particular file with Startle raw data within a directory
        Output 1   : dictionary with columns (trial names) as keys and Vmax as values
        Output 2   : dictionary with columns (trial names) as keys and Tmax as values
        
        
        Description : 
        1) Calculate Vmax and Tmax for each trial
        2) save 'no stimulus' trials in an exclusion list
        3) add low vmax trials (<cut_criterion) derived from stimulus-on conditions in another exclusion list"""
    
    Vmax_dic = {}
    Tmax_dic = {}
                                                           
    for column in bigdf.columns :    
        bigdf[column][6:-1] = bigdf[column][6:-1].astype(int)         # the numbers are by default strings; Here we change them to int 
        highest_V = bigdf[column][8:-1].max()                         # then we find the V max of every column, always searching from row 8 onwards since that's the stimulus onset
        highest_T = (bigdf[bigdf[column] == highest_V].index[0]) - 8  # and the respective T value
        
        if bigdf[column].iloc[5] != ' No_stimulus' :                  # in case it was a stimulus-off trial the trial will be added to an exclusion list (see else)
            both_criteria_satisfied = True                            # in order for V value to be accepted as a final V value, it has to be large enough. Moreover, its relevant Tvalue shouldn't be
            if highest_V <= cut_criterion_V :                         # if the vmax was lower than cut_criterion_V, then we save the column's name to a specific exclusion list
                all_too_low[file].append(column)                       
                both_criteria_satisfied = False                       # This will prevent adding the V value to the final values, even if its T relevant value satisfies the criteria
            if highest_T >= cut_criterion_T:                          # if the Tmax was higher than cut_criterion_T, then we save the column's name to a specific exclusion list
                all_too_slow[file].append(column)                     # mind the difference, this is all_too_slow, not all_too_low
                both_criteria_satisfied = False                       # This will prevent adding the V value to the final values, even if it satisfies the criteria
            if both_criteria_satisfied == True :                      # this will only take place if the 2 if statements above are not activated
                Vmax_dic[column] = highest_V
        else :                                                        # if it was a no stimulus trial, then we save its name to another exclusion list
            no_stim_excl[file].append(column) 
            
    return Vmax_dic

In [ ]:
def ExcludeNoisyOnset (df,file,cutms=5,cutsize=20,cutcounter=3):
    """Input 1: dataframe
       Input 2: str indicating a particular file with Startle raw data within a directory
       Input 3: int 
       Input 4: int 
       Input 5: int
       global all_premature_resp : dict with raw data file names as keys (str) and dict as VALUES. 
                dict key : 'b10_c1.csv'
                     value {'Trial1_Mouse1': 73, 'Trial1_Mouse2': 49, ... } 
       
       Locates and excludes trials wherein noteworthy motion was detected too early following stimulus onset.
       Criterion : if between 0 and 0+cutms miliseconds there is movement equal or higher than cutsize (V value) at least 
       cutcounter times, then the trial is added to exclusion list. Note: 0 is stimulus onset. Note that in the script for
       time responses, the last for loop does not exist here. It is implemented in FindPeaks()"""
 
    all_columns = df.columns[:-1]                                                                                           # -1 to get rid of 'miliseconds' column
    for column_index in range(len(all_columns)):
        curr_trial = all_columns[column_index]
        if (curr_trial in no_stim_excl[file]) or (curr_trial in all_too_low[file]) or (curr_trial in all_too_slow[file]) :
            continue
        else :                                                                                                              # another criterion could be if any response in this window is 25 or more
            counter = 0
            for point in list(df[curr_trial][8:8+cutms]) :                                                                  # remember that 8 corresponds to 0 ms. Point is a point in time
                if point >= cutsize :
                    counter += 1
                    if counter > cutcounter :
                        all_premature_resp[file].append(curr_trial)
                        break  
    
    for prem_trial in all_premature_resp[file]:                                                                             # delete premature trials from file with final V values
        del all_final_V[file][prem_trial]            

In [ ]:
def VerifyExclusions (all_final_V,file) :
    """Input 1: dict with raw data file names as keys (str) and dict as VALUES. 
                dict key : 'b10_c1.csv'
                     value {'Trial1_Mouse1': 73, 'Trial1_Mouse2': 49, ... }
       Input 2: str indicating a particular file with Startle raw data within a directory """
    
    global no_stim_excl
    global all_too_low
    global all_too_slow
    global all_premature_resp
    
    counter = 0 
    for trial in all_final_V[file] :
        if (trial in no_stim_excl[file]) or (trial in all_too_low[file]) or (trial in all_premature_resp[file]) or (trial in all_too_slow[file]):
            print('Error detected. Some excluded trials have not been filtered out.')
            counter += 1
        else: 
            pass
    if counter == 0 :
        print ("\n All filtering criteria have been applied successfully !")

## Main Code

In [ ]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
path = r'C:\Users\angdid\Desktop\Thib'
files = [f for f in listdir(path) if isfile(join(path, f))]
all_final_V = {}
all_too_low = {}  # Excluded trials from all raw data in a folder where the default Vmax was below minimum criteria 
all_too_slow = {}
all_dfs = {} 
all_premature_resp = {}
no_stim_excl = {} 

for file in files :
    print('loading ', file)
    big_df = pd.read_csv(join(path,file))
    big_df.columns = ColumNames()
    all_too_low[file] = []
    all_too_slow[file] = []
    all_premature_resp[file]=[]
    no_stim_excl[file] = []
    
    # Default Vmax and Tmax except fore the cases where no stimulus is present or Vmax is too low (see parameters)
    all_final_V[file] = Find_Maxima_And_Exclude(big_df,file)    
    df = big_df.copy(deep = True) 
    ExcludeNoisyOnset(df,file)
    all_dfs[file] = df

VerifyExclusions(all_final_V,file)

# Create excel with final values  for statistical analysis 

Trials of 120dB do not include pre-pulse. We use them to calculate the reaction to the stimulus per se (control). However, we don't take into all 120db trials in our experiment. Only some of them ( the "sandwiched" ones ) These are the trials 7,16..66 as indicated in list "sandwiched" below. 

In [ ]:
# an expected mistake here is same mouse id in different trials because the user hasnt changed it on Startle

In [ ]:
def IDLocation (big_df):
    """Input : big dataframe (without excluded trials)
       Output: int indicating the row wherein the mouse id is in the raw data file
       
       For some reason, sometimes the mouse IDs are in different columns in Startle's raw data files.
       Could be because there is a Subject and an ID column. And I was usually using the Subject columns to write IDs.
       This is to locate the right row provided that mouse id has 6 digits"""
    
    for y in range(6):
        lista = (list(big_df[big_df.columns[0:4]].iloc[y])) #Look at 6 first rows of first 4 col, not only 1st col, because 1st might be No entry
        for i in lista:
            try :
                int(i)
                if re.findall(r'\d{6}',i):
                    return y
            except:
                pass

In [ ]:
def GleandB (Vmax,file,sandwiched,N_animals=4) :
    """ 
       Input 1: Dict with trials (str) as keys and Vvalue of that trial as value. These are the all_final_V values of the file.
       Input 1 example : {'Trial1_Mouse1' 312
                          'Trial1_Mouse2': 474
                          'Trial1_Mouse3': 415
                          'Trial1_Mouse4': 410
                          'Trial2_Mouse1': 278
                          ... etc ...
       Input 2: str indicating a particular file with Startle raw data within a directory
       Input 3: list with 120dB trials (indicating points in an experiment's trial sequence) that we are solely interested in
       Input 4: int representing the number of animals in an experiment (normally we use all 4 boxes).
       
       Output : Dict -> Keys: str representing real mouse id
                        Values: Dict -> keys: dB conditions (str) 
                                        value: list with all T (or V, depending on input 5) values observed for each dB condition
                                        
       Output example : {'  423952': {'Startle_120dB': [14, 14, 14, 15, 14, 15, 14, 14, 14, 14, 14, 14], 
                                       'PPI_85dB': [16, 15, 15, 16, 16, 14, 15, 14, 14, 14], 
                                       'PPI_80dB': [15, 15, 15, 15, 15, 14, 15, 14], 
                                       'PPI_75dB': [14, 15, 15, 14, 14, 15, 16, 14, 16, 15]}, '  
                            423456': ... etc ...
      
      Non-returned output : appending real mouse ids (as values) to a dict with numbers corresponding to boxes as keys.
                            This dict is the cage_and_mouse variable.
      
      This function does nothing more than rearranging already existent information. See Input1 and Output example """
       
    curr_df = all_dfs[file]                                                                                               # isolate the dataframe of interest
    id_row = IDLocation(all_dfs[file])                                                                                    # locate the row where the ID is 
    boxid = [(f'Mouse{curr_df[column].loc[6]}',curr_df[column].loc[id_row]) for column in curr_df[curr_df.columns[8:12]]] #6th row always contains the boxnumber and by default box 1 corresponds to Mouse1. Id row can vary but it contains the real id. Here, we create (boxnumber, real id) 

    # Mouse1 means mouse in box 1 and so on...
    
    dB = { 'Mouse1' : {'Startle_120dB': [] , 'PPI_85dB' : [] , 'PPI_80dB' : [], 'PPI_75dB' : [] },
           'Mouse2' : {'Startle_120dB': [] , 'PPI_85dB' : [] , 'PPI_80dB' : [], 'PPI_75dB' : [] }, 
           'Mouse3' : {'Startle_120dB': [] , 'PPI_85dB' : [] , 'PPI_80dB' : [], 'PPI_75dB' : [] },
           'Mouse4' : {'Startle_120dB': [] , 'PPI_85dB' : [] , 'PPI_80dB' : [], 'PPI_75dB' : [] }
         }
    
    for trial in Vmax:  
        condition = df[trial].loc[5]                                                             # Can be either No stimulus or x_NdB, located always at 5th row in the dataframe. Here we are using the name of the trial from Vmax dictionary to locate it from the df
        mouse_number = trial.split('_')[1]                                                       # for instance, isolate Mouse3 from 'Trial1_Mouse3'
        valdB = re.findall('\d+', condition)[0]                                                  # For unknown reason, raw data have either one or two whitespaces before condition itself
        ppiORstartle = condition.split('_')[0].strip()                                           #for instance, isolate 'PPI' from ' PPI_85dB' or '  PPI_85dB'. Or isolate Startle from 'Startle_120dB' 
        stimtype = f'{ppiORstartle}_{valdB}dB'                                                   # note that str has been stripped. Thus there is no whitespace in the beginning
        if ('Startle' in stimtype) and (df[trial].loc[7] not in sandwiched):                     # if there is a 120dB condition but is not in the ones we study. [7] gives the number of the trial
            pass
        else :
            dB[mouse_number][stimtype].append(Vmax[trial])                                       # Vmax[trial] to take the value of the trial, which is a key of Vmax
                                                                                                 # Now we want to change Mouse1,Mouse2... to real id names. We'll use boxid which is a list with [MouseN,realid], where N is a number from 1 to 4
    for cageN,real_id in boxid:                                                                  # What we are doing is taking everything after Mouse1,Mouse2... and adding it to a newly created key in dB dictionary. This key has the real id number
        dB[real_id] = dB.pop(cageN)                                                              # pop the key with its value and create a new mouse ID key with this key and value
        cage_and_mouse[int(cageN[-1])].append(real_id)                                           # This is to know to which out of the 4 boxes does the ID belong to when we'll be creating final excel columns. Note that if there were more than 9 boxes, regex should be used instead of -1
        
    return dB

In [ ]:
def MixandMean (dB,output_df) :
    """Input 1: Dict -> Keys: str representing real mouse id
                        Values: Dict -> keys: dB conditions (str) 
                                        values: list with all V (or T) values observed for each condition
                For more clarity see the Output exampled of GleandB()
                
       Input 2: empty df with real ids as columns and ['120dB', '75-85dB','%PPI', 'Filtered_Trials_PPI','Filtered_Trials_Non_PPI','Box'] as indices
       Output : filled df with real ids as columns and ['120dB', '75-85dB','%PPI', 'Filtered_Trials_PPI','Filtered_Trials_Non_PPI','Box'] as indices"""
    
    for mouse in dB.keys() :
        PPlen = len (dB[mouse]['PPI_85dB']) + len (dB[mouse]['PPI_80dB']) + len (dB[mouse]['PPI_75dB'])
        PPsum = sum (dB[mouse]['PPI_85dB']) + sum (dB[mouse]['PPI_80dB']) + sum (dB[mouse]['PPI_75dB'])
        try :
            PPAvg = round (PPsum / PPlen, ndigits=2)
            NoPPAvg = round (st.mean(map(float,dB[mouse]['Startle_120dB'])), ndigits=2)
        except (StatisticsError,ZeroDivisionError) as e:
            excluded_mice.append(mouse) #in case of absence of values
            continue
          
        output_df[mouse]['120dB'] = NoPPAvg
        output_df[mouse]['75-85dB'] = PPAvg
        output_df[mouse]['%PPI'] = (1 - (PPAvg/NoPPAvg)) * 100 
        output_df[mouse]['Filtered_Trials_PPI']= PPlen
        output_df[mouse]['Filtered_Trials_Non_PPI']= len(dB[mouse]['Startle_120dB'])
        
    
    return output_df

In [ ]:
def ExcludeFewTrials (PPI_accepted_N=13,NonPPI_accepted_N=4) :
    """Excludes animals where only few trials were filtered in (Excel is a dataframe and will eventually become the output to excel)"""
    for ID in excel.columns:
        if excel[ID][3] >= PPI_accepted_N and excel[ID][4] >= NonPPI_accepted_N :                          # See excel. 3 is for Filtered_Trials_PPI, 4 for NonPPI
            pass
        else:
            print('Low number of values for mouse {mouse}. This will not be included in final excel file'.format(mouse=excel[ID].name))
            excluded_mice.append(excel[ID].name)
            del excel[ID]

In [ ]:
all_file_ids = []
sandwiched = [7,16,19,23,27,33,38,45,47,54,59,66]
excluded_mice = []                                                                                                 # append takes place in mixandmean()
excel = pd.DataFrame(index = ['120dB', '75-85dB','%PPI', 'Filtered_Trials_PPI','Filtered_Trials_Non_PPI','Box'])
cage_and_mouse = {1:[],2:[],3:[],4:[]}                                                                             # real mouse ids will be appended here in GleandB()

for file in files: 
    dB = GleandB(all_final_V[file],file,sandwiched)
    output_df = pd.DataFrame(columns = dB.keys(), index = ['120dB', '75-85dB','%PPI', 'Filtered_Trials_PPI','Filtered_Trials_Non_PPI','Box'] )
    output_df = MixandMean(dB,output_df)
    for column in output_df.columns :
        if 'Entry' not in column:
            excel[column]= output_df[column]
            all_file_ids.append(column.strip())

for key,value in cage_and_mouse.items():
    value = [ident for ident in value if "Entry" not in ident]
    cage_and_mouse[key] = value
    
for col_id_str in excel.columns:
    for dic_index in range(1,len(cage_and_mouse)+1) :                                                               # that should just be 4 when using 4 boxes
        if col_id_str in cage_and_mouse[dic_index]:
            excel[col_id_str].loc['Box'] = dic_index

ExcludeFewTrials()                                                                                                  # excludes mice based on the number of filtered_in values for PPI and NonPPI
excel = excel.T  #transpose
excel.to_excel ('C:\\Users\\angdid\\Desktop\\test.xlsx')

#can add a code to test for same ids in different files